In [ ]:
from neuron import h
import numpy as np
import time
import brainstate
import braintools
import brainunit as u
import matplotlib.pyplot as plt
import jax.numpy as jnp
import braincell
from scipy.interpolate import interp1d
import os

h.load_file('stdrun.hoc')
s = u.siemens / u.cm ** 2
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
brainstate .environ.set(precision=64)

In [ ]:
# from neuron import h, gui
# import numpy as np
# import matplotlib.pyplot as plt

# h.load_file("stdrun.hoc")

# # 通用参数设置
# duration = 100  # 模拟时长 (ms)
# dt = 0.025       # 时间步长 (ms)
# sections = 3     # 3个section

# def setup_model(connect_type):
#     """创建模型并返回section列表"""
#     sections = []
#     for i in range(3):
#         sec = h.Section(name=f'sec{i}')
#         sec.nseg = 1      # 每个section仅1个segment
#         sec.diam = 10     # 直径 (μm)
#         sec.L = 100       # 长度 (μm)
#         sec.Ra = 100      # 轴向电阻 (Ω·cm)
#         sec.cm = 1        # 膜电容 (μF/cm²)
#         sec.insert('pas') # 被动属性
#         sec.g_pas = 0.001 # 漏电导 (S/cm²)
#         sec.e_pas = -70   # 静息电位 (mV)
#         sections.append(sec)
    
#     # 连接方式

#     if connect_type == 'unbranched':
#         sections[1].connect(sections[0](1))  # sec1(0) -> sec0(1)
#         sections[2].connect(sections[1](1))  # sec2(0) -> sec1(1)
#     elif connect_type == 'branched':
#         sections[1].connect(sections[0](1))  # sec1(0) -> sec0(1)
#         sections[2].connect(sections[0](1))  # sec2(0) -> sec0(1)
    
#     return sections

# # 运行模拟函数
# def run_simulation(connect_type):
    
#     sections = setup_model(connect_type)
    
#     # 在sec0中心注入电流脉冲
#     if connect_type == 'line':
#         stim = h.IClamp(sections[1](0.5))
#         stim.delay = 20
#         stim.dur = 40
#         stim.amp = 0.1  # nA
#     else:
#         stim = h.IClamp(sections[0](0.5))
#         stim.delay = 20
#         stim.dur = 40
#         stim.amp = 0.1  # nA

    
#     # 记录数据
#     t = h.Vector().record(h._ref_t)
#     v_rec = [h.Vector().record(sec(0.5)._ref_v) for sec in sections]
#     h.finitialize(-70)
#     h.continuerun(duration)
#     return np.array(t), np.array(v_rec)[0], np.array(v_rec)[1], np.array(v_rec)[2]

# # 运行两种模型
# t, v0_line, v1_line, v2_line = run_simulation('line')
# _, v0_star, v1_star, v2_star = run_simulation('star')

# # 计算电压差
# diff_v0 = v1_line - v0_star
# diff_v1 = v0_line - v1_star
# diff_v2 = v2_line - v2_star

# # 绘图
# plt.figure(figsize=(12, 8))

# # 链式连接电压
# plt.subplot(2,1,1)
# plt.plot(t, v0_line, label='soma (line)')
# plt.plot(t, v1_line, label='dend1 (line)')
# plt.plot(t, v2_line, label='dend2 (line)')
# plt.title('Chain Connection Voltages')
# plt.ylabel('Voltage (mV)')
# plt.legend()

# # 电压差异对比
# plt.subplot(2,1,2)
# plt.plot(t, diff_v0, '--', label='soma difference')
# plt.plot(t, diff_v1, '--', label='dend1 difference')
# plt.plot(t, diff_v2, '--', label='dend2 difference')
# plt.title('Voltage Differences (Line - Star)')
# plt.xlabel('Time (ms)')
# plt.ylabel('Δ Voltage (mV)')
# plt.legend()

# plt.tight_layout()
# plt.show()

## Create multicompartment cell in Neuron

In [ ]:
##params
L_soma = 10
L_axon = 10
diam_soma = 10
diam_axon = 1
NUM = 10

In [ ]:
class Neuron:
    def __init__(self, num, connection_mode, connection = None):
        self.num = num
        self.sec = []
        self.create_sections()
        self.set_params()
        if connection_mode == 'line':
            self.connect_line()
        elif connection_mode == 'star':
            self.connect_star()
        else:
            self.process_connection(connection)
        
        
    def create_sections(self):
        for i in range(self.num):
            sec = h.Section(name=f'sec_{i}')
            self.sec.append(sec)
    
    def connect_line(self):
        for i in range(1, self.num):
            self.sec[i].connect(self.sec[i - 1](1))

    def process_connection(self, connection):
        for pair in connection:
            if len(pair) == 3:
                id1, id2, site = pair
                if int(id2)!= -1:
                    self.sec[int(id1)].connect(self.sec[int(id2)](site))

    
    def connect_star(self):
        for i in range(1, self.num):
            self.sec[i].connect(self.sec[0](1))

    def set_params(self):
        Ra = 100 * np.ones(self.num)
        cm = 1 * np.ones(self.num)
        diam = diam_axon * np.ones(self.num)
        #diam[0] = diam_soma
        L = L_axon * np.ones(self.num)
        #L[0] = L_soma

        for i, sec in enumerate(self.sec):
            sec.Ra = Ra[i]
            sec.cm = cm[i]
            sec.diam = diam[i]
            sec.L = L[i]
            
            sec.insert('pas')
            sec(0.5).pas.g = 0.001
            sec(0.5).pas.e = -65
            if i ==0:
                sec(0.5).v = -55
            if i!=0:
                sec(0.5).v = -55
            # if i == 0:
            #     sec.insert('hh')
            # else:
            #     sec.insert('pas')
            #     sec(0.5).pas.g = 0.001
            #     sec(0.5).pas.e = -65
    
               

## run
def NeuronRun(cell, stim, tstop, dt, v_init):
    ## create record vector
    t_vec = h.Vector()
    t_vec.record(h._ref_t)
    v_vecs = []

    for sec in cell.sec:   
        v_vec = h.Vector()
        v_vec.record(sec(0.5)._ref_v)
        v_vecs.append(v_vec)

    ## simulation
    h.celsius = 6.3
    h.dt = dt
    h.v_init = v_init
    h.finitialize()
    h.continuerun(tstop)
    return t_vec, v_vecs

## different stim func
def step_stim(cell, delay, dur, amp):
    stim = h.IClamp(cell.sec[0](0.5))
    stim.delay = delay
    stim.dur = dur
    stim.amp = amp
    return stim

def sine_stim(cell, amp, freq, phase, tstop, dt):
    t = np.arange(0, tstop, dt)  
    I = amp * np.sin(2 * np.pi * freq * t + phase) 

    t_vec = h.Vector(t)
    I_vec = h.Vector(I)
    ic = h.IClamp(cell.sec[0](0.5))
    ic.delay = 0  
    ic.dur = tstop 
    I_vec.play(ic._ref_amp, t_vec)
    
    return ic, t_vec, I_vec

def random_sine_combination(t, num_sines, seed=None):
    if seed is not None:
        np.random.seed(seed)
    result = np.zeros_like(t)
    for _ in range(num_sines):
        amplitude = np.random.uniform(-1, 1)
        frequency = np.random.uniform(0.1, 1)
        phase = np.random.uniform(0, 2 * np.pi)
        result += amplitude * np.sin(2 * np.pi * frequency * t + phase)
    return result

def random_sine_stim(cell, seed, num_sines, tstop, dt):
    t = np.arange(0, tstop, dt)  
    I = random_sine_combination(t, num_sines, seed)

    t_vec = h.Vector(t)
    I_vec = h.Vector(I)
    ic = h.IClamp(cell.sec[0](0.5))
    ic.delay = 0  
    ic.dur = tstop 
    I_vec.play(ic._ref_amp, t_vec)

    return ic, t_vec, I_vec

def random_stim(cell, seed, upper, lower, tstop, dt):
    np.random.seed(seed)
    t = np.arange(0, tstop, dt)  
    I = np.random.uniform(lower, upper, size=len(t))

    t_vec = h.Vector(t)
    I_vec = h.Vector(I)
    ic = h.IClamp(cell.sec[0](0.5))
    ic.delay = 0  
    ic.dur = tstop 
    I_vec.play(ic._ref_amp, t_vec)

    return ic, t_vec, I_vec

## Create multicompartment cell in Braincell

In [ ]:
class INa(braincell.channel.SodiumChannel):
    def __init__(self, size, g_max):
        super().__init__(size)

        self.g_max = brainstate.init.param(g_max, self.varshape)

    def init_state(self, V, Na: braincell.IonInfo, batch_size: int = None):
        self.m = braincell.DiffEqState(self.m_inf(V))
        self.h = braincell.DiffEqState(self.h_inf(V))

    def compute_derivative(self, V, Na: braincell.IonInfo):
        self.m.derivative = (self.m_alpha(V) * (1 - self.m.value) - self.m_beta(V) * self.m.value) / u.ms
        self.h.derivative = (self.h_alpha(V) * (1 - self.h.value) - self.h_beta(V) * self.h.value) / u.ms

    def current(self, V, Na: braincell.IonInfo):
        return self.g_max * self.m.value ** 3 * self.h.value * (Na.E - V)

    # m channel
    m_alpha = lambda self, V: 1. / u.math.exprel(-(V / u.mV + 40.) / 10.)  # nan
    m_beta = lambda self, V: 4. * u.math.exp(-(V / u.mV + 65.) / 18.)
    m_inf = lambda self, V: self.m_alpha(V) / (self.m_alpha(V) + self.m_beta(V))

    # h channel
    h_alpha = lambda self, V: 0.07 * u.math.exp(-(V / u.mV + 65.) / 20.)
    h_beta = lambda self, V: 1. / (1. + u.math.exp(-(V / u.mV + 35.) / 10.))
    h_inf = lambda self, V: self.h_alpha(V) / (self.h_alpha(V) + self.h_beta(V))


class IK(braincell.channel.PotassiumChannel):
    def __init__(self, size, g_max):
        super().__init__(size)
        self.g_max = brainstate.init.param(g_max, self.varshape)

    def init_state(self, V, K: braincell.IonInfo, batch_size: int = None):
        self.n = braincell.DiffEqState(self.n_inf(V))

    def compute_derivative(self, V, K: braincell.IonInfo):
        self.n.derivative = (self.n_alpha(V) * (1 - self.n.value) - self.n_beta(V) * self.n.value) / u.ms

    def current(self, V, K: braincell.IonInfo):
        return self.g_max * self.n.value ** 4 * (K.E - V)

    n_alpha = lambda self, V: 0.1 / u.math.exprel(-(V / u.mV + 55.) / 10.)
    n_beta = lambda self, V: 0.125 * u.math.exp(-(V / u.mV + 65.) / 80.)
    n_inf = lambda self, V: self.n_alpha(V) / (self.n_alpha(V) + self.n_beta(V))

In [ ]:
class Braincell(braincell.neuron.MultiCompartment):
    def __init__(self,  size, connection, Ra, cm, diam, L, gl, Gl=0, El=-65, V_init=-55, solver = 'exp_euler'):
        super().__init__(
            size=size,
            connection= connection,
            Ra= Ra * u.ohm * u.cm,
            cm= cm * u.uF / u.cm ** 2,
            diam=diam * u.um,
            L= L * u.um,
            V_th=20. * u.mV,
            V_initializer=brainstate.init.Constant( V_init * u.mV),
            spk_fun=brainstate.surrogate.ReluGrad(),
            solver = solver,
            Gl = Gl * s, 
            El = El * u.mV
        )
        # gna = np.zeros(len(L))
        # gna[0] = 0.120
        # gk =  np.zeros(len(L))
        # gk[0]= 0.036
        # gl = 0.001*np.ones(len(L))
        # gl[0]= 0.0003
        # el = -65 * np.ones(len(L))
        # el[0] = -54.3
        
        self.IL = braincell.channel.IL(self.varshape, E=  -65 * u.mV, g_max= gl * s)
        
        # self.na =  braincell.ion.SodiumFixed(self.varshape, E= 50. * u.mV)
        # self.na.add(INa = INa(self.varshape, g_max= gna * s))

        # self.k = braincell.ion.PotassiumFixed(self.varshape, E= -77. * u.mV)
        # self.k.add(IK =IK(self.varshape, g_max= gk * s))
        
    def step_run(self, t, inp):
        with brainstate.environ.context(t=t):
            self.update(inp)
            return self.V.value
        
def con_line(num_compartment):
    connection = [[i, i + 1] for i in range(num_compartment - 1)]
    return connection

def con_star(num_compartment):
    connection = [[0, i + 1] for i in range(num_compartment - 1)]
    return connection

def Braincell_create(num, connection_mode, gl, connection_data = None, Gl=0, El= -65, V_init=-55, solver = 'cn_rk4'): 
    size = (1, num)
    Ra = 100 * np.ones(num)
    cm = 1 * np.ones(num)
    #cm[0] = 2
    diam = diam_axon * np.ones(num)
    #diam[0] = diam_soma
    L = L_axon * np.ones(num)
    #L[1] =2*L_axon
    #L[0] = L_soma
    ## connection set
    if connection_mode == 'line':
        connection = con_line(num)
    elif connection_mode == 'star':
        connection = con_star(num)
    else:
        connection = connection_data

    return Braincell(size, connection, Ra, cm, diam, L, gl, Gl=Gl, El =El, V_init=V_init, solver=solver)  

## run 
def BraincellRun(neu,I, dt):
    ## time
    brainstate.environ.set(dt= dt * u.ms)
    times = u.math.arange(I.shape[0]) * brainstate.environ.get_dt()
    ## init and reset
    neu.init_state()
    neu.reset_state()
    ## run
    vs = brainstate.compile.for_loop(neu.step_run, times, I)
    return times.to_decimal(u.ms), vs.to_decimal(u.mV)

## different input func
def step_input(num, dur, amp, dt):
    brainstate.environ.set(dt= dt * u.ms)
    value = jnp.zeros((len(dur),num))
    for i in range(len(value)):
        value = value.at[i, 0].set(amp[i])
        # value = value.at[i, 0].add(GE)
        # value = value.at[i, 1].set(GE)
        # value = value.at[i, 2].set(GE)


    I = braintools.input.section_input(values=value, durations= dur * u.ms) * u.nA
    return I

def sine_input(num, amp, freq, phase, tstop, dt):
    time = np.arange(0, tstop, dt)
    I = np.zeros((len(time),num)) * u.nA
    I[:,0] = amp * np.sin(2 * np.pi * freq * time + phase) * u.nA
    return I

def random_sin_input(num, seed, num_sines, tstop, dt):
    time = np.arange(0, tstop, dt)
    I = np.zeros((len(time),num)) * u.nA
    I[:,0] =  random_sine_combination(time, num_sines, seed)*u.nA
    return I

def random_input(num, seed, upper, lower, tstop, dt):
    np.random.seed(seed)
    time = np.arange(0, tstop, dt)
    I = np.zeros((len(time),num)) * u.nA
    I[:,0] = np.random.uniform(lower, upper, size=len(time)) * u.nA
    return I


## Simulation in Neuron and Braincell

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm

# 定义 3x3 矩阵 A
A = np.array([[-5/3, 2/3, 1],
              [1/3, -1/3, 0],
              [1, 0, -1]])*10**3/4


A = np.array([[-2, 1, 1],
              [1, -1, 0],
              [1, 0, -1]])*10**3/4/(1+1/2)  ## L[0]=10

A = np.array([[-2/3, 1/3, 1/3],
              [2/3, -2/3, 0],
              [2/3, 0, -2/3]])*10**3/4/(1+2/3)  ## L[0]=20

A = np.array([[-2/10, 1/10, 1/10],
              [1/2.5, -1/2.5, 0],
              [1/2.5, 0, -1/2.5]])*10**3/4/(1+4/5)  ## L[0]=40

A = np.array([[-2/6, 1/6, 1/6],
              [1/2, -1/2, 0],
              [1/2, 0, -1/2]])*10**3/4/(1+3/4)  ## L[0]=30

A = np.array([[-(1/5+1/1.5), 1/5, 1/1.5],
              [1/5, -1/5, 0],
              [1/1.5, 0, -1/1.5]])*10**3/4  ## L[1] = 20 ,diam[1] =0.5 主电阻均分


# A = np.array([[-(1/(6.5)+8/9), 1/(6.5), 8/9],
#               [1/(6.5), -1/(6.5), 0],
#               [8/9, 0, -8/9]])*10**3/4  ## L[1] = 20 ,diam[1] =0.5 主电阻按截面积均分 4:1 

# A = np.array([[-(1/(8/37)+1/3), 1/(8/37), 1/3],
#               [1/(8/37), -1/(8/37), 0],
#               [1/3, 0, -1/3]])*10**3/4  ## L[1] = 20 ,diam[1] =0.5 主电阻按截面积均分 4:1  对掉一下主电阻分裂开的顺序


# A = np.array([[-(1/8.5+16/17), 1/8.5, 16/17],
#               [1/8.5, -1/8.5, 0],
#               [16/17, 0, -16/17]])*10**3/4  ## L[1] = 20 ,diam[1] =0.5 主电阻按子电阻比例均分 8:1  对掉顺序

# A = np.array([[-(16/73+1/5), 16/73, 1/5],
#               [16/73, -16/73, 0],
#               [1/5, 0, -1/5]])*10**3/4  ## L[1] = 20 ,diam[1] =0.5 主电阻按子电阻比例均分 8:1 


# A = np.array([[-(1/5.5+4/5), 1/5.5, 4/5],
#               [1/5.5, -1/5.5, 0],
#               [4/5, 0, -4/5]])*10**3/4  ## L[1] = 20 ,diam[1] =0.5 主电阻按子电阻直径比例均分 2:1 

# A = np.array([[-(1/4.5+1), 1/4.5, 1],
#               [1/4.5, -1/4.5, 0],
#               [1, 0, -1]])*10**3/4 ## L[1] = 20 ,diam[1] =0.5 返璞归真

A = np.array([[-2/1.5, 1/1.5, 1/1.5],
              [2/1.5, -2/1.5, 0],
              [2/1.5, 0, -2/1.5]])*10**3/4 ## L[0] = 10, D[0] = 2 

A = np.array([[-2/4, 1/4, 1/4],
              [1, -1, 0],
              [1, 0, -1]])*10**3/4 ## L[0] = 20, D[0] = 2 

A = np.array([[-1, 1/2, 1/2],
              [1/4, -1/4, 0],
              [1/4, 0, -1/4]])*10**3/4 ## L[12] = 20

A = np.array([[-16/9, 8/9, 8/9],
              [4/9, -4/9, 0],
              [4/9, 0, -4/9]])*10**3/4 ## D[12] = 2 

A = np.array([[-(2/5+4/5), 2/5, 4/5],
              [1/5, -1/5, 0],
              [4/5, 0, -4/5]])*10**3/4 ## L[1] = 20 

A = np.array([[-2, 2, 0, 0 ],
              [2, -6, 2, 2],
              [0, 2, -2, 0],
              [0, 2, 0, -2]])*10**3/4  ## 空节点

A = np.array([[-2, 2, 0, 0 ],
              [1, -5, 2, 2],
              [0, 2, -2, 0],
              [0, 2, 0, -2]])*10**3/4  ## 空节点 L[0] = 20

A = np.array([[-2/5, 1/5, 1/5],
              [2/5, -6/5, 4/5],
              [2/5, 4/5, -6/5] ])*10**3/4  ## 空节点 L[0] = 20
 
 
V0 = np.array([-65, -55, -55])  # 初始条件
v_values = np.linspace(0, 0.1, 10001)  # 取 0 到 2 的时间范围

V0_values = [] 
V1_values = [] 
V2_values = [] 

for v in v_values:
    E = expm(A * v)  # 计算 e^(A*v)
    V = E @ V0  # 计算 V(v)
    V0_values.append(V[0])  # 记录 V[0] 分量
    V1_values.append(V[1]) 
    V2_values.append(V[2]) 

# 绘图
plt.figure(figsize=(8, 5))
plt.plot(v_values, V0_values, label='Exp0')
plt.plot(v_values, V1_values, label='Exp1')
plt.plot(v_values, V2_values, label='Exp2')
plt.xlabel('Time v')
plt.ylabel(r'$V_0(v)$')
plt.title('Time Evolution of $V_0$')
plt.legend()
plt.grid()
plt.show()


In [ ]:
h.secondorder = 0

In [ ]:
loaded_params = np.load('golgi_morphology.npz')
#connection_golgi = loaded_params['connection']
connection_golgi = np.load('golgi_connection.npy')
connection_test = connection_golgi
neuron = Neuron(317, connection_mode = None, connection= connection_test)
stim = step_stim(neuron, delay=0, dur=10, amp=0.00)
t_vec, v_vecs = NeuronRun(neuron, stim=stim, tstop=0.2, dt=0.0001, v_init=-55)
for i in range(1):
    plt.plot(t_vec, v_vecs[i], label=f'sec_{i}')
#plt.plot(v_values, V0_values, label='Exp', color='b')
plt.legend()
plt.xlabel('Time (ms)')
plt.ylabel('Voltage (mV)')
plt.title('Voltage(NEURON)')
plt.grid(True)
plt.show()

In [ ]:
# num = 3
# neuron = Neuron(num, connection_mode = 'star')
# #neuron.sec[0].diam = 2
# # neuron.sec[1].diam = 2
# # neuron.sec[2].diam = 2
# neuron.sec[0].L = 10
# stim = step_stim(neuron, delay=0, dur=10, amp=0.00)
# #stim = sine_stim(cell=neuron, amp=1, freq=1, phase=0, tstop=10, dt=0.001)
# #stim = random_stim(cell=neuron, seed=1, upper=1, lower=0, tstop=10, dt=0.1)
# #stim = random_sine_stim(cell=neuron, seed=2, num_sines=100, tstop=11, dt=0.001)
# t_vec, v_vecs = NeuronRun(neuron, stim=stim, tstop=0.1, dt=0.0001, v_init=-55)

# for i in range(3):
#     plt.plot(t_vec, v_vecs[i], label=f'sec_{i}')
# plt.plot(v_values, V0_values, label='Exp0')
# #plt.plot(v_values, V2_values, label='Exp1')
# #plt.plot(v_values, V0_values, label='Exp2')

# plt.legend()
# plt.xlabel('Time (ms)')
# plt.ylabel('Voltage (mV)')
# plt.title('Voltage(NEURON)')
# plt.grid(True)
# plt.show()

In [ ]:
El = -65; Gl=0.001; num =317
V_init = -55 *np.ones(num); V_init[0] = -65
cell = Braincell_create(num=num, connection_mode=None, connection_data= connection_test, gl = 0, Gl=0, El = El, V_init=V_init, solver='cn_rk4')
I = step_input(num=num, dur=[0.2,0,0], amp=[0.0,0.0,0], dt=0.001)  #GE = 0.1*El*Gl*np.pi,
#I = sine_input(num=num, amp=1, freq= 0.3, phase=0, tstop= 10, dt=0.1)
#I = random_input(num=num, seed=1, upper=1, lower=0, tstop=10, dt=0.01)
#I = random_sin_input(num=num, seed=1, num_sines=10, tstop=10, dt=0.01)
t1, v1 = BraincellRun(neu=cell, I=I, dt=0.001)

for i in range(1):
    plt.plot(t1,v1[:,:,i])
    plt.plot(t_vec, v_vecs[i], label=f'sec_{i}')
#plt.plot(v_values, V0_values, label='Exp', color='b')
plt.legend()
plt.xlabel('Time (ms)')
plt.ylabel('Voltage (mV)')
plt.title('Voltage(Braincell)')
plt.grid(True)
plt.show()

In [ ]:
num =3
El = -65; Gl=0.001
V_init = -55 *np.ones(num); V_init[0] = -65
cell = Braincell_create(num=num, connection_mode='star', gl = 0, Gl=0, El = El, V_init=V_init, solver='cn_rk4')
I = step_input(num=num, dur=[0.2,0,0], amp=[0.0,0.0,0], dt=0.001)  #GE = 0.1*El*Gl*np.pi,
#I = sine_input(num=num, amp=1, freq= 0.3, phase=0, tstop= 10, dt=0.1)
#I = random_input(num=num, seed=1, upper=1, lower=0, tstop=10, dt=0.01)
#I = random_sin_input(num=num, seed=1, num_sines=10, tstop=10, dt=0.01)
t1, v1 = BraincellRun(neu=cell, I=I, dt=0.001)

for i in range(num):
    plt.plot(t1,v1[:,:,i])
plt.legend()
plt.xlabel('Time (ms)')
plt.ylabel('Voltage (mV)')
plt.title('Voltage(Braincell)')
plt.grid(True)
plt.show()

## Compare the error

### 1. check error of spliiting ～ O(dt)

In [ ]:
num_dt = 6
dt_list = [0.001 * 2 ** i for i in range(num_dt)]
error_1_list = []
error_2_list = []
error_inf_list = []
v_list = [] 

connection_golgi_chunk = connection_golgi
num = 317

## standerd curve 
neuron = Neuron(num, connection_mode =None, connection=connection_golgi_chunk)
stim = step_stim(neuron, delay=0, dur=100, amp=0.0)
#stim = sine_stim(cell=neuron, amp=1, freq=0.3, phase=0, tstop=11, dt=0.001)
#stim = random_stim(cell=neuron, seed=1, upper=1, lower=0, tstop=10, dt=0.1)
#stim = random_sine_stim(cell=neuron, seed=1, num_sines=10, tstop=11, dt=0.001)

t_vec, v_vecs = NeuronRun(neuron, stim=stim, tstop=1, dt=0.001, v_init=-55)
t_vec = np.array(t_vec);t_vec = t_vec[:-1000]
v_vecs = np.array(v_vecs);v_vecs = v_vecs[:,:-1000]

## true solution
# cell = Braincell_create(num=num, connection_mode='line', gl=0.001, Gl=0, El=-65, solver='exp_euler')
# I = step_input(num=num, dur=[0,10,0], amp=[0,0.01,0],  dt = 0.001)
# t_vec, v_vec = BraincellRun(neu=cell, I=I, dt = 0.001)
# t_vec = np.array(t_vec);t_vec = t_vec[:-1000]
# v_vec = np.array(v_vecs);v_vec = v_vec[:-1000,:,0]
v_list.append(v_vecs[0])

for i in range(num_dt):
    El = -65; Gl=0.001
    V_init = -55*np.ones(num); V_init[0] = -55
    cell = Braincell_create(num=num, connection_mode= None, connection_data=connection_golgi_chunk,gl= 0, Gl=Gl, V_init=V_init,  solver='cn_rk4')
    I = step_input(num=num, dur=[0,100,0.], amp=[0,0.0,0], dt=dt_list[i])
    #I = sine_input(num=num, amp=1, freq= 0.3, phase=0, tstop= 11, dt=dt_list[i])
    #I = random_input(num=num, seed=1, upper=1, lower=0, tstop=10, dt=dt_list[i])
    #I = random_sin_input(num=num, seed=1, num_sines=10, tstop=11, dt=dt_list[i])
    t1, v1 = BraincellRun(neu=cell, I=I, dt=dt_list[i])
    v1 = np.roll(v1, 1, axis=0)
    v1[0] = V_init

    ## interpolation for the same length
    f = interp1d(t1, v1[:,:,0].reshape(-1), kind='linear')
    v_list.append(f(t_vec))
    ## error
    error_1_list.append(np.linalg.norm(v_vecs[0] -f(t_vec),1)/len(v_vecs[0]))
    error_2_list.append(np.linalg.norm(v_vecs[0] -f(t_vec),2)/len(v_vecs[0]))
    error_inf_list.append(np.linalg.norm(v_vecs[0] -f(t_vec),np.inf))

# log_e
log_dt = np.log10(dt_list)
log_error_1 = np.log10(error_1_list)
log_error_2 = np.log10(error_2_list)
log_error_inf = np.log10(error_inf_list)

# ref line
x0 = log_dt[0]
y0 = log_error_1[0]
b = y0 - 1 * x0  
reference_line_y = 1 * log_dt + b
plt.plot(log_dt, reference_line_y, linestyle='--', color='red', label='Slope = 1')

# plot
plt.plot(log_dt, log_error_1, marker='o', label='1-norm error')
#plt.plot(log_dt, log_error_2, marker='s', label='2-norm error')
#plt.plot(log_dt, log_error_inf, marker='^', label='Inf-norm error')

#plt.title('Errors vs dt(CN+Rk4)')
plt.xlabel('log(dt)')
plt.ylabel('log(Error)')
plt.grid(True)
plt.legend()

plt.gca().set_aspect('equal', adjustable='box')
plt.show()

## dt compare
plt.figure(figsize=(8, 6))
for i in range(num_dt+1):
    if i == 0:
        plt.plot(t_vec,v_list[i], linewidth=3,color = 'black', label = 'NEURON')
    elif i<=20:
        plt.plot(t_vec,v_list[i], alpha = 0.9, label = f'dt={dt_list[i-1]}ms')
    plt.legend()
plt.xlabel('Time (ms)')
plt.ylabel('Voltage (mV)')
#plt.title('Voltage')
plt.grid(True)
plt.show()


## NEURON convergence

In [ ]:
num_dt = 12
dt_list = [0.00001 * 2 ** i for i in range(num_dt)]
error_1_list = []
error_2_list = []
error_inf_list = []
v_list = [] 

connection_golgi_chunk = connection_golgi[:2]
num = 317

## standerd curve 
neuron = Neuron(num, connection_mode = None, connection= connection_golgi_chunk)
stim = step_stim(neuron, delay=0, dur=0.2, amp=0.0)

t_vec, v_vecs = NeuronRun(neuron, stim=stim, tstop=0.2, dt=0.000001, v_init=-65)
t_vec = np.array(t_vec);t_vec = t_vec[:-100000]
v_vecs = np.array(v_vecs);v_vecs = v_vecs[:,:-100000]

v_list.append(v_vecs[0])

for i in range(num_dt):

    neuron_1 = Neuron(num, connection_mode = None, connection= connection_golgi_chunk)
    stim_1 = step_stim(neuron, delay=0, dur=0.2, amp=0.01)
    t1, v1 = NeuronRun(neuron_1, stim=stim_1, tstop=0.2, dt=dt_list[i], v_init=-65)
    t1 = np.array(t1) ; v1 =  np.array(v1) 
    ## interpolation for the same length
    f = interp1d(t1, v1[0,:].reshape(-1), kind='linear')
    v_list.append(f(t_vec))
    ## error
    error_1_list.append(np.linalg.norm(v_vecs[0] -f(t_vec),1)/len(v_vecs[0]))
    error_2_list.append(np.linalg.norm(v_vecs[0] -f(t_vec),2)/len(v_vecs[0]))
    error_inf_list.append(np.linalg.norm(v_vecs[0] -f(t_vec),np.inf))

# log_e
log_dt = np.log10(dt_list)
log_error_1 = np.log10(error_1_list)
log_error_2 = np.log10(error_2_list)
log_error_inf = np.log10(error_inf_list)

# ref line
x0 = log_dt[0]
y0 = log_error_1[0]
b = y0 - 1 * x0  
reference_line_y = 1 * log_dt + b
plt.plot(log_dt, reference_line_y, linestyle='--', color='red', label='Slope = 1')

# plot
plt.plot(log_dt, log_error_1, marker='o', label='1-norm error')
#plt.plot(log_dt, log_error_2, marker='s', label='2-norm error')
#plt.plot(log_dt, log_error_inf, marker='^', label='Inf-norm error')

#plt.title('Errors vs dt(CN+Rk4)')
plt.xlabel('log(dt)')
plt.ylabel('log(Error)')
plt.grid(True)
plt.legend()

plt.gca().set_aspect('equal', adjustable='box')
plt.show()

## dt compare
plt.figure(figsize=(8, 6))
for i in range(num_dt+1):
    if i == 0:
        plt.plot(t_vec,v_list[i], linewidth=3,color = 'black', label = 'NEURON')
    elif i<=20:
        plt.plot(t_vec,v_list[i], alpha = 0.9, label = f'dt={dt_list[i-1]}ms')
    plt.legend()
plt.xlabel('Time (ms)')
plt.ylabel('Voltage (mV)')
#plt.title('Voltage')
plt.grid(True)
plt.show()


## Briancell convergence

In [ ]:
num_dt = 12
dt_list = [0.00001 * 2 ** i for i in range(num_dt)]
error_1_list = []
error_2_list = []
error_inf_list = []
v_list = [] 

connection_golgi_chunk = connection_golgi[:2]
num = 317

El = -65; Gl=0.001
V_init = -55*np.ones(num); V_init[0] = -65

cell_1 = Braincell_create(num=num, connection_mode=None, connection_data= connection_golgi_chunk, gl= 0, Gl=0, V_init=V_init,  solver='cn_rk4')
I = step_input(num=num, dur=[0,0.2,0.], amp=[0,0.0,0], dt=0.00001)
t_vec, v_vec = BraincellRun(neu=cell_1, I=I, dt=0.00001)
v_vec= np.roll(v_vec, 1, axis=0)
v_vec[0] = V_init

t_vec = t_vec[:-10000]
v_vec = v_vec[:-10000,:,0].reshape(-1)
v_list.append(v_vec)

for i in range(num_dt):

    cell = Braincell_create(num=num, connection_mode=None, connection_data= connection_golgi_chunk, gl= 0, Gl=0, V_init=V_init,  solver='cn_rk4')
    I = step_input(num=num, dur=[0,0.2,0.], amp=[0,0.0,0], dt=dt_list[i])
    t1, v1 = BraincellRun(neu=cell, I=I, dt=dt_list[i])
    v1 = np.roll(v1, 1, axis=0)
    v1[0] = V_init

    ## interpolation for the same length
    f = interp1d(t1, v1[:,:,0].reshape(-1), kind='linear')
    v_list.append(f(t_vec))
    ## error
    error_1_list.append(np.linalg.norm(v_vec -f(t_vec),1)/len(v_vec))
    error_2_list.append(np.linalg.norm(v_vec -f(t_vec),2)/len(v_vec))
    error_inf_list.append(np.linalg.norm(v_vec -f(t_vec),np.inf))

# log_e
log_dt = np.log10(dt_list)
log_error_1 = np.log10(error_1_list)
log_error_2 = np.log10(error_2_list)
log_error_inf = np.log10(error_inf_list)

# ref line
x0 = log_dt[1]
y0 = log_error_1[1]
b = y0 - 1 * x0  
reference_line_y = 1 * log_dt + b
plt.plot(log_dt, reference_line_y, linestyle='--', color='red', label='Slope = 1')

# plot
plt.plot(log_dt, log_error_1, marker='o', label='1-norm error')
#plt.plot(log_dt, log_error_2, marker='s', label='2-norm error')
#plt.plot(log_dt, log_error_inf, marker='^', label='Inf-norm error')

#plt.title('Errors vs dt(CN+Rk4)')
plt.xlabel('log(dt)')
plt.ylabel('log(Error)')
plt.grid(True)
plt.legend()

plt.gca().set_aspect('equal', adjustable='box')
plt.show()

## dt compare
plt.figure(figsize=(8, 6))
for i in range(num_dt+1):
    if i == 0:
        plt.plot(t_vec,v_list[i], linewidth=3,color = 'black', label = 'Braincell')
    elif i<=9:
        plt.plot(t_vec,v_list[i], alpha = 0.9, label = f'dt={dt_list[i-1]}ms')
    plt.legend()
plt.xlabel('Time (ms)')
plt.ylabel('Voltage (mV)')
#plt.title('Voltage')
plt.grid(True)
plt.show()



 ## CN不是L-稳定，导致在分裂法中误差较大
 1.切换分裂的顺序，误差较大

In [ ]:
# import numpy as np
# from scipy.linalg import expm
# import matplotlib.pyplot as plt

# # 定义矩阵 A
# A = np.array([[-4.5454544,  4.5454544],
#            [ 4.5454544e-01, -4.5454544e-01]])*10**2

# # 初始条件
# y0 = np.array([-60, -65])

# # 时间区间和时间步长
# t0 = 0
# T = 20
# h = 0.01
# # 时间点
# t = np.arange(t0, T, h)
# gl = 1
# el = -65
# I_ext =  1000*np.ones((len(t), len(y0)))/np.pi
# I_ext[:,1] = 0

# # 矩阵指数法
# y_expm = np.zeros((len(t), len(y0)))
# y_expm[0] = y0
# exp_Ah = expm(A * h)
# for n in range(len(t) - 1):
#     y_star =  y_expm[n] + h * ( -gl*(y_expm[n]-el) + I_ext[n])
#     y_expm[n + 1] = np.dot(exp_Ah, y_star)

# # 一阶隐式欧拉方法
# def ieu_euler(mode):
    
#     y_ieuler = np.zeros((len(t), len(y0)))
#     y_ieuler[0] = y0
#     y_star = np.zeros((len(t), len(y0)))
#     y_star[0] =  y0
#     I = np.eye(len(A))
#     if mode == 0:
#         for n in range(len(t) - 1):
#             M = I - h * A
#             y_star[n + 1] = y_ieuler[n] + h * (-gl*(y_ieuler[n]-el) + I_ext[n])
#             y_ieuler[n + 1] = np.linalg.solve(M,y_star[n + 1])
#     elif mode == 1:
#         for n in range(len(t) - 1):
#             M = I - h * A
#             y_star[n + 1] = np.linalg.solve(M,y_ieuler[n])
#             y_ieuler[n + 1] = y_star[n + 1] + h * (-gl*(y_star[n + 1]-el) + I_ext[n])
#     return y_star

# # 二阶隐式中点法
# y_imidpoint = np.zeros((len(t), len(y0)))
# y_imidpoint[0] = y0
# y_imidpoint_star = np.zeros((len(t), len(y0)))
# y_imidpoint_star[0] =  y0
# I = np.eye(len(A))
# for n in range(len(t) - 1):
#     M1 = I - (h / 2) * A
#     M2 = I + (h / 2) * A
#     y_imidpoint_star[n+1] = y_imidpoint[n] + h * (-gl*(y_imidpoint[n]-el) + I_ext[n])
#     b = np.dot(M2, y_imidpoint_star[n+1])
#     y_imidpoint[n + 1] = np.linalg.solve(M1, b)


# y_ieuler_0 = ieu_euler(0)
# y_ieuler_1 = ieu_euler(1)

# # 绘制结果
# plt.figure(figsize=(12, 8))
# plt.plot(t, v_exact[:, 0], label="v1_Exp", linestyle="-", linewidth=2)
# plt.plot(t, v_exact[:, 1], label="v2_Exp", linestyle="-", linewidth=2)
# # for i in range(num):
# #     plt.plot(t_vec, v_vecs[i], label=f'sec_{i}')
# #plt.plot(t, y_expm[:, 0], label='y1 (expm)', linestyle='-')
# #plt.plot(t, y_expm[:, 1], label='y2 (expm)', linestyle='-')
# plt.plot(t, y_ieuler_0[:, 0], label='y1 (1st Euler_ieu)', linestyle='--')
# plt.plot(t, y_ieuler_0[:, 1], label='y2 (1st Euler_ieu)', linestyle='--')
# plt.plot(t, y_ieuler_1[:, 0], label='y1 (1st Ieu_euler)', linestyle='--')
# plt.plot(t, y_ieuler_1[:, 1], label='y2 (1st Ieu_euler)', linestyle='--')

# #plt.plot(t, y_imidpoint_star[:, 0], label='y1 (2nd star)', linestyle='-.')
# #plt.plot(t, y_imidpoint_star[:, 1], label='y2 (2nd star)', linestyle='-.')
# #plt.plot(t, y_imidpoint[:, 0], label='y1 (2nd Implicit Midpoint)', linestyle='-.')
# #plt.plot(t, y_imidpoint[:, 1], label='y2 (2nd Implicit Midpoint)', linestyle='-.')
# plt.xlabel('Time')
# plt.ylabel('Solution')
# plt.title('Comparison of expm, 1st Implicit Euler and 2nd Implicit Midpoint')
# plt.legend()
# plt.grid(True)
# plt.show()

In [ ]:
# import numpy as np
# from scipy.linalg import expm, inv
# import matplotlib.pyplot as plt

# # 参数设置
# g12 = 4.5454544 * 10**2  # 连接电导
# g21 = 4.5454544 * 10  
# gL = 1  # 漏电导
# EL = -65  # 漏电平衡电位
# I1 = 1000 / np.pi  # 注入电流
# I2 = 0  # 第二个房室没有外部电流

# # 矩阵 A
# A = np.array([[-g12 - gL, g12],
#               [g21, -g21 - gL]])

# # 常数向量 b
# b = np.array([gL * EL + I1, gL * EL + I2])

# # 初始条件
# v0 = np.array([-60, -65])

# # 时间参数
# t0 = 0
# T = 20
# h = 0.01
# t = np.arange(t0, T, h)

# # 计算矩阵指数 e^(At)
# exp_At = np.array([expm(A * ti) for ti in t])

# # 计算解析解 v(t) = e^(At) v0 + A^(-1) (e^(At) - I) b
# I = np.eye(A.shape[0])  # 单位矩阵
# A_inv = inv(A)  # 计算 A 的逆

# v_exact = np.zeros((len(t), len(v0)))
# for i, ti in enumerate(t):
#     v_exact[i] = exp_At[i] @ v0 + A_inv @ (exp_At[i] - I) @ b

# # 使用隐式欧拉法求解（验证解析解）
# v_euler = np.zeros((len(t), len(v0)))
# v_euler[0] = v0
# M = I - h * A  # 预计算 M 矩阵
# for n in range(len(t) - 1):
#     v_star = v_euler[n] + h * b  # 计算逼近解
#     v_euler[n + 1] = np.linalg.solve(M, v_star)

# # 画图对比
# plt.figure(figsize=(10, 6))
# plt.plot(t, v_exact[:, 0], label="v1_Exp", linestyle="-", linewidth=2)
# plt.plot(t, v_exact[:, 1], label="v2_Exp", linestyle="-", linewidth=2)
# plt.plot(t, v_euler[:, 0], label="v1", linestyle="--", linewidth=1.5)
# plt.plot(t, v_euler[:, 1], label="v2", linestyle="--", linewidth=1.5)

# plt.legend()
# plt.grid()
# plt.show()


In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt

# # 定义参数
# dt = 0.01  # 时间步长
# T = 0.2    # 总模拟时间
# N = int(T / dt)  # 总步数

# lambda_A = 500  # 控制扩散刚性
# alpha = 1.0    # 控制非线性项
# v0 = 0.5       # 初值
# times = np.linspace(0, T, N+1)

# # 精确解（仅适用于线性情况，用于误差对比）
# def exact_solution(t, v0, lambda_A, alpha):
#     return v0 * np.exp(-lambda_A * t)  # 近似精确解

# # 定义非线性项
# def g(v):
#     return alpha * (1 - v) + 0.2

# def rk4_step(v, dt):
#     k1 = g(v)
#     k2 = g(v + 0.5 * dt * k1)
#     k3 = g(v + 0.5 * dt * k2)
#     k4 = g(v + dt * k3)
#     return v + (dt / 6) * (k1 + 2*k2 + 2*k3 + k4)

# # 初始化数值解
# v_be = np.zeros(N+1)  # 向后欧拉 + RK4
# v_cn = np.zeros(N+1)  # 梯形公式 + RK4
# v_be[0] = v0
# v_cn[0] = v0

# # 数值计算
# for i in range(N):
#     # 先用 RK4 计算非线性项
#     v_temp_be = rk4_step(v_be[i], dt)
#     v_temp_cn = rk4_step(v_cn[i], dt)
    
#     # 向后欧拉计算扩散项
#     v_be[i+1] = v_temp_be / (1 + lambda_A * dt)
    
#     # 梯形公式计算扩散项
#     v_cn[i+1] = (v_temp_cn - 0.5 * dt * lambda_A * v_cn[i]) / (1 + 0.5 * dt * lambda_A)

# # 计算误差
# exact = exact_solution(times, v0, lambda_A, alpha)
# error_be = np.abs(v_be - exact)
# error_cn = np.abs(v_cn - exact)

# # 绘图比较
# plt.figure(figsize=(8,5))
# plt.plot(times, error_be, label='Backward Euler + RK4', linestyle='dashed')
# plt.plot(times, error_cn, label='Crank-Nicolson + RK4', linestyle='dotted')
# plt.xlabel("Time")
# plt.ylabel("Absolute Error")
# plt.legend()
# plt.title("Error Comparison: Backward Euler vs. Crank-Nicolson with Operator Splitting")
# plt.show()


In [ ]:
def compute_eigenvalues(N, k):
    diag_main = np.full(N, -2*k)
    diag_main[0] = -k  
    diag_main[-1] = -k
    diag_off = np.full(N-1, k)
    A = np.diag(diag_main) + np.diag(diag_off, k=1) + np.diag(diag_off, k=-1)
    return np.linalg.eigvals(A)

print(compute_eigenvalues(3, 2.5))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Construct matrix A
A = np.array([[-0.0001,  0.0001,  0.    ,  0.    ,  0.    ,  0.    ],
           [ 0.0001, -0.0002,  0.0001,  0.    ,  0.    ,  0.    ],
           [ 0.    ,  0.0001, -0.0002,  0.0001,  0.    ,  0.    ],
           [ 0.    ,  0.    ,  0.0001, -0.0002,  0.0001,  0.    ],
           [ 0.    ,  0.    ,  0.    ,  0.0001, -0.0002,  0.0001],
           [ 0.    ,  0.    ,  0.    ,  0.    ,  0.0001, -0.0001]]) * 10**7

A = np.array([
    [-251, 250,   0],
    [250, -501, 250],
    [0, 250, -251]
], dtype=np.float64)


# Compute eigenvalues and eigenvectors, then sort by real part
eigvals, eigvecs = np.linalg.eig(A)
idx = np.argsort(eigvals.real)
eigvals = eigvals[idx]
eigvecs = eigvecs[:, idx]

print("Eigenvalues:", eigvals)
print("Eigenvectors:\n", eigvecs)

# Define initial state V0 and external current I_ext
V0 = np.array([ -65, -65, -65])
GE = -65*np.ones(len(V0))
I_ext = np.zeros(len(V0))
I_ext[0] = 100/np.pi  # current injected only in the first compartment
I_ext = I_ext + GE

dt = 0.1
T_total = 10.0  # Total simulation time in seconds
n_steps = int(T_total / dt)

# Compute initial projection coefficients in the eigenvector basis
a0 = np.dot(eigvecs.T.conj(), V0)
init_amp = np.abs(a0)  # initial amplitudes in each mode

# Prepare arrays to record mode multipliers for two update steps:
mode_multipliers_intermediate = np.zeros((n_steps + 1, len(V0)))
mode_multipliers_final = np.zeros((n_steps + 1, len(V0)))
mode_multipliers_intermediate[0, :] = 1.0
mode_multipliers_final[0, :] = 1.0

# Prepare array to record voltage V for each time step
V_record = np.zeros((n_steps + 1, len(V0)))
V_record[0, :] = V0.copy()

# Set up initial state V
V = V0.copy()

# Precompute matrices for CN update:
I_mat = np.eye(len(V0))
CN_left = I_mat - 1 * dt * A
CN_right = I_mat + 0. * dt * A

# Time stepping using splitting method:
# Step 1: External current update (forward Euler)
# Step 2: Diffusion update (Crank-Nicolson)
for i in range(1, n_steps + 1):
    # Step 1: External current update
    V_temp = V + dt * I_ext
    coeff_temp = np.dot(eigvecs.T.conj(), V_temp)
    amp_temp = np.abs(coeff_temp)
    multiplier_temp = amp_temp #/ init_amp
    mode_multipliers_intermediate[i, :] = multiplier_temp

    # Step 2: Diffusion update using Crank-Nicolson
    V_next = np.linalg.solve(CN_left, CN_right @ V_temp)
    V = V_next.copy()
    coeff_final = np.dot(eigvecs.T.conj(), V)
    amp_final = np.abs(coeff_final)
    multiplier_final = amp_final #/ init_amp
    mode_multipliers_final[i, :] = multiplier_final

    # Record final voltage V at this time step
    V_record[i, :] = V.copy()

# Time vector for plotting
time = np.linspace(0, T_total, n_steps + 1)

# Plot evolution of mode multipliers for both intermediate and final states in one figure
plt.figure(figsize=(10, 6))
for mode in range(len(V0)):
    # Plot intermediate state (after external current update) with dashed line
    plt.plot(time, mode_multipliers_intermediate[:, mode], linestyle='--', label=f"Mode {mode+1} (Intermediate)")
    # Plot final state (after diffusion update) with solid line
    plt.plot(time, mode_multipliers_final[:, mode], linestyle='-', label=f"Mode {mode+1} (Final)")
plt.xlabel("Time (s)")
plt.ylabel("Multiplier (relative to initial projection)")
#plt.title("Evolution of Mode Multipliers (Splitting Method)")
plt.legend()
plt.grid(True)
plt.show()

# Plot evolution of V (voltage) for each compartment
plt.figure(figsize=(10, 6))
for comp in range(len(V0)):
    plt.plot(time, V_record[:, comp], linestyle='-', label=f"Compartment {comp+1}")
plt.xlabel("Time (s)")
plt.ylabel("Voltage V")
#plt.title("Evolution of Voltage V Over Time")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm

# Construct matrix A
A = np.array([
    [-251, 250,   0],
    [250, -501, 250],
    [0, 250, -251]
], dtype=np.float64)

# Define initial state V0 and external current I_ext
V0 = np.array([-65, -65, -65])
GE = -65 * np.ones(len(V0))
I_ext = np.zeros(len(V0))
I_ext[0] = 100 / np.pi  # current injected only in the first compartment
B = I_ext + GE

dt = 0.1
T_total = 10.0  # Total simulation time in seconds
n_steps = int(T_total / dt)

# Compute eigenvalues and eigenvectors, then sort by real part
eigvals, eigvecs = np.linalg.eig(A)
idx = np.argsort(eigvals.real)
eigvals = eigvals[idx]
eigvecs = eigvecs[:, idx]

print("Eigenvalues:", eigvals)
print("Eigenvectors:\n", eigvecs)

# Compute initial projection coefficients in the eigenvector basis
a0 = np.dot(eigvecs.T.conj(), V0)
init_amp = np.abs(a0)  # initial amplitudes in each mode

# Prepare arrays to record mode multipliers for two update steps:
mode_multipliers_intermediate = np.zeros((n_steps + 1, len(V0)))
mode_multipliers_final = np.zeros((n_steps + 1, len(V0)))
# mode_multipliers_intermediate[0, :] = 1.0
# mode_multipliers_final[0, :] = 1.0

# Prepare array to record voltage V for each time step
V_record = np.zeros((n_steps + 1, len(V0)))
V_record[0, :] = V0.copy()

# Set up initial state V
V = V0.copy()

# Precompute matrices for CN update:
I_mat = np.eye(len(V0))
CN_left = I_mat - 0.5 * dt * A
CN_right = I_mat + 0.5 * dt * A

# Time stepping using splitting method:
# Step 1: External current update (forward Euler)
# Step 2: Diffusion update (Crank-Nicolson)
for i in range(1, n_steps + 1):
    # Step 1: External current update
    V_temp = V + dt * B
    coeff_temp = np.dot(eigvecs.T.conj(), V_temp)
    amp_temp = np.abs(coeff_temp)
    multiplier_temp = amp_temp  # / init_amp
    mode_multipliers_intermediate[i, :] = multiplier_temp

    # Step 2: Diffusion update using Crank-Nicolson
    V_next = np.linalg.solve(CN_left, CN_right @ V_temp)
    V = V_next.copy()
    coeff_final = np.dot(eigvecs.T.conj(), V)
    amp_final = np.abs(coeff_final)
    multiplier_final = amp_final  # / init_amp
    mode_multipliers_final[i, :] = multiplier_final

    # Record final voltage V at this time step
    V_record[i, :] = V.copy()

# Calculate analytical solution
time = np.linspace(0, T_total, n_steps + 1)
V_analytical = np.zeros((n_steps + 1, len(V0)))
for i, t in enumerate(time):
    exp_At = expm(A * t)
    integral_term = np.linalg.inv(A) @ (exp_At - np.eye(len(V0))) @ B
    V_analytical[i, :] = exp_At @ V0 + integral_term

# Calculate mode multipliers for analytical solution
mode_multipliers_analytical = np.zeros((n_steps + 1, len(V0)))
for i in range(n_steps + 1):
    coeff_analytical = np.dot(eigvecs.T.conj(), V_analytical[i, :])
    amp_analytical = np.abs(coeff_analytical)
    multiplier_analytical = amp_analytical  # / init_amp
    mode_multipliers_analytical[i, :] = multiplier_analytical

# Calculate errors
error_mode_multipliers_intermediate = np.abs(mode_multipliers_intermediate - mode_multipliers_analytical)
error_mode_multipliers_final = np.abs(mode_multipliers_final - mode_multipliers_analytical)
error_V = np.abs(V_record - V_analytical)

# Plot evolution of mode multipliers for both intermediate and final states in one figure
plt.figure(figsize=(10, 6))
for mode in range(len(V0)-1):
    # Plot intermediate state (after external current update) with dashed line
    plt.plot(time, mode_multipliers_intermediate[:, mode], linestyle='--', label=f"Mode {mode + 1} (Intermediate)")
    # Plot final state (after diffusion update) with solid line
    plt.plot(time, mode_multipliers_final[:, mode], linestyle='-', label=f"Mode {mode + 1} (Final)")
plt.xlabel("Time (s)")
plt.ylabel("Multiplier (relative to initial projection)")
plt.legend()
plt.grid(True)
plt.show()

# Plot evolution of V (voltage) for each compartment
plt.figure(figsize=(10, 6))
for comp in range(len(V0)):
    plt.plot(time, V_record[:, comp], linestyle='-', label=f"Compartment {comp + 1}")
plt.xlabel("Time (s)")
plt.ylabel("Voltage V")
plt.legend()
plt.grid(True)
plt.show()

# Plot error of mode multipliers
plt.figure(figsize=(10, 6))
for mode in range(len(V0)-1):
    # Plot intermediate state (after external current update) with dashed line
    #plt.plot(time, error_mode_multipliers_intermediate[:, mode], linestyle='--', label=f"Mode {mode + 1} (Intermediate)")
    # Plot final state (after diffusion update) with solid line
    plt.plot(time, error_mode_multipliers_final[:, mode], linestyle='-', label=f"Mode {mode + 1} (Final)")
plt.xlabel("Time (s)")
plt.ylabel("Error in Mode Multipliers")
plt.title("Error in Mode Multipliers (Numerical vs Analytical)")
plt.legend()
plt.grid(True)
plt.show()

# Plot error of V
plt.figure(figsize=(10, 6))
for comp in range(len(V0)):
    if comp ==0:
        plt.plot(time, error_V[:, comp], linestyle='-', label=f"Compartment {comp + 1}")
plt.xlabel("Time (s)")
plt.ylabel("Error in Voltage V")
plt.title("Error in Voltage V (Numerical vs Analytical)")
plt.legend()
plt.grid(True)
plt.show()
    

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm

# 构造矩阵 A
A = np.array([
    [-251, 250, 0],
    [250, -501, 250],
    [0, 250, -251]
], dtype=np.float64)

# 定义初始状态 V0 和外部电流 I_ext
V0 = np.array([-65, -65, -65])
GE = -65 * np.ones(len(V0))
I_ext = np.zeros(len(V0))
I_ext[0] = 100 / np.pi  # 仅在第一个隔室注入电流
B = I_ext + GE

T_total = 10.0  # 总模拟时间（秒）
dt_precise = 1e-5  # 用于计算精确解的小步长
n_steps_precise = int(T_total / dt_precise)

# 计算精确解
time_precise = np.linspace(0, T_total, n_steps_precise + 1)
V_analytical_precise = np.zeros((n_steps_precise + 1, len(V0)))
for i, t in enumerate(time_precise):
    exp_At = expm(A * t)
    integral_term = np.linalg.inv(A) @ (exp_At - np.eye(len(V0))) @ B
    V_analytical_precise[i, :] = exp_At @ V0 + integral_term

# 计算精确解的特征向量和特征值
eigvals_precise, eigvecs_precise = np.linalg.eig(A)
idx = np.argsort(eigvals_precise.real)
eigvals_precise = eigvals_precise[idx]
eigvecs_precise = eigvecs_precise[:, idx]

# 计算精确解的模式乘数
mode_multipliers_analytical_precise = np.zeros((n_steps_precise + 1, len(V0)))
for i in range(n_steps_precise + 1):
    coeff_analytical = np.dot(eigvecs_precise.T.conj(), V_analytical_precise[i, :])
    amp_analytical = np.abs(coeff_analytical)
    multiplier_analytical = amp_analytical
    mode_multipliers_analytical_precise[i, :] = multiplier_analytical

dt_values = np.logspace(-4, -1, 10)  # 不同的 dt 值
steady_state_error_mode = []
steady_state_error_V = []

for dt in dt_values:
    n_steps = int(T_total / dt)

    # 计算特征值和特征向量，然后按实部排序
    eigvals, eigvecs = np.linalg.eig(A)
    idx = np.argsort(eigvals.real)
    eigvals = eigvals[idx]
    eigvecs = eigvecs[:, idx]

    # 计算初始投影系数
    a0 = np.dot(eigvecs.T.conj(), V0)
    init_amp = np.abs(a0)  # 每个模式的初始振幅

    # 准备记录模式乘数和电压的数组
    mode_multipliers_intermediate = np.zeros((n_steps + 1, len(V0)))
    mode_multipliers_final = np.zeros((n_steps + 1, len(V0)))
    V_record = np.zeros((n_steps + 1, len(V0)))
    V_record[0, :] = V0.copy()

    V = V0.copy()

    # 预计算 Crank - Nicolson 更新所需的矩阵
    I_mat = np.eye(len(V0))
    CN_left = I_mat - 1 * dt * A
    CN_right = I_mat + 0 * dt * A

    # 时间步长循环
    for i in range(1, n_steps + 1):
        # 步骤 1：外部电流更新（前向欧拉法）
        V_temp = V + dt * B
        coeff_temp = np.dot(eigvecs.T.conj(), V_temp)
        amp_temp = np.abs(coeff_temp)
        multiplier_temp = amp_temp
        mode_multipliers_intermediate[i, :] = multiplier_temp

        # 步骤 2：扩散更新（Crank - Nicolson 法）
        V_next = np.linalg.solve(CN_left, CN_right @ V_temp)
        V = V_next.copy()
        coeff_final = np.dot(eigvecs.T.conj(), V)
        amp_final = np.abs(coeff_final)
        multiplier_final = amp_final
        mode_multipliers_final[i, :] = multiplier_final

        V_record[i, :] = V.copy()

    # 插值获取对应精确解时间点的值
    def interpolate_data(data, time, target_time):
        return np.array([np.interp(target_time, time, data[:, i]) for i in range(data.shape[1])]).T

    V_analytical_interp = interpolate_data(V_analytical_precise, time_precise, np.linspace(0, T_total, n_steps + 1))
    mode_multipliers_analytical_interp = interpolate_data(mode_multipliers_analytical_precise, time_precise,
                                                          np.linspace(0, T_total, n_steps + 1))

    # 计算误差
    error_mode_multipliers_intermediate = np.abs(mode_multipliers_intermediate - mode_multipliers_analytical_interp)
    error_mode_multipliers_final = np.abs(mode_multipliers_final - mode_multipliers_analytical_interp)
    error_V = np.abs(V_record - V_analytical_interp)

    # 计算稳态误差
    steady_state_error_mode.append(error_mode_multipliers_final[-1, :])
    steady_state_error_V.append(error_V[-1, :])

steady_state_error_mode = np.array(steady_state_error_mode)
steady_state_error_V = np.array(steady_state_error_V)

# 绘制模式的稳态误差
plt.figure(figsize=(10, 6))
for mode in range(len(V0)):
    plt.loglog(dt_values, steady_state_error_mode[:, mode], label=f"Mode {mode + 1}")
plt.xlabel("dt")
plt.ylabel("Steady State Error in Mode Multipliers")
plt.title("Steady State Error in Mode Multipliers vs dt")
plt.legend()
plt.grid(True)
plt.show()

# 绘制电压的稳态误差
plt.figure(figsize=(10, 6))
for comp in range(len(V0)):
    plt.loglog(dt_values, steady_state_error_V[:, comp], label=f"Compartment {comp + 1}")
plt.xlabel("dt")
plt.ylabel("Steady State Error in Voltage V")
plt.title("Steady State Error in Voltage V vs dt")
plt.legend()
plt.grid(True)
plt.show()
    

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Construct matrix A
A = np.array([[-0.0001,  0.0001,  0.    ,  0.    ,  0.    ,  0.    ],
           [ 0.0001, -0.0002,  0.0001,  0.    ,  0.    ,  0.    ],
           [ 0.    ,  0.0001, -0.0002,  0.0001,  0.    ,  0.    ],
           [ 0.    ,  0.    ,  0.0001, -0.0002,  0.0001,  0.    ],
           [ 0.    ,  0.    ,  0.    ,  0.0001, -0.0002,  0.0001],
           [ 0.    ,  0.    ,  0.    ,  0.    ,  0.0001, -0.0001]]) * 10**7

A = np.array([
    [-250, 250,   0],
    [250, -500, 250],
    [0, 250, -250]
], dtype=np.float64)



# Compute eigenvalues and eigenvectors, then sort by real part
eigvals, eigvecs = np.linalg.eig(A)
idx = np.argsort(eigvals.real)
eigvals = eigvals[idx]
eigvecs = eigvecs[:, idx]

print("Eigenvalues:", eigvals)
print("Eigenvectors:\n", eigvecs)

# Define initial state V0 and external current I_ext
V0 = np.array([ -65, -65, -65])
I_ext = np.zeros(len(V0))
I_ext[0] = 100/np.pi # current injected only in the first compartment

dt = 0.1
T_total = 10.0  # Total simulation time in seconds
n_steps = int(T_total / dt)

# Precompute CN matrices for the full system including I_ext:
I = np.eye(len(V0))
# Pure CN update:
# (I - dt/2 * A)*V_{n+1} = (I + dt/2 * A)*V_n + dt * I_ext
CN_left = I - 0.5* dt * A
CN_right = I + 0.5 * dt * A

# Record V evolution for plotting V curves
V_record = np.zeros((n_steps + 1, len(V0)))
V_record[0, :] = V0.copy()

# Compute initial projection coefficients in the eigenvector basis
coeff0 = np.dot(eigvecs.T.conj(), V0)
init_amp = np.abs(coeff0)  # initial amplitudes in each mode

# Record the evolution of mode multipliers relative to initial amplitudes
mode_multipliers = np.zeros((n_steps + 1, len(V0)))
mode_multipliers[0, :] = 1.0  # initial multiplier = 1

# Set initial state V
V = V0.copy()

# Time stepping using pure CN update (jointly treating A and I_ext)
for i in range(1, n_steps + 1):
    V_next = np.linalg.solve(CN_left, CN_right @ V + dt * I_ext)
    V = V_next.copy()
    
    # Record V for plotting V curves
    V_record[i, :] = V
    
    # Compute projection coefficients in the eigenvector basis
    coeff = np.dot(eigvecs.T.conj(), V)
    amp = np.abs(coeff)
    multiplier = amp #/ init_amp  # relative to initial amplitude (initial = 1)
    mode_multipliers[i, :] = multiplier

# Plot evolution of mode multipliers (relative to initial projection)
time = np.linspace(0, T_total, n_steps + 1)
plt.figure(figsize=(10, 6))
for mode in range(len(V0)-1):
    plt.plot(time, mode_multipliers[:, mode], label=f"Mode {mode+1}")
plt.xlabel("Time (s)")
plt.ylabel("Multiplier (relative to initial)")
#plt.title("Pure Im Method")
plt.legend()
plt.grid(True)
plt.show()

# Plot evolution of V (solution) over time for each compartment
plt.figure(figsize=(10, 6))
for comp in range(len(V0)):
    plt.plot(time, V_record[:, comp], label=f"Compartment {comp+1}")
plt.xlabel("Time (s)")
plt.ylabel("V (solution)")
#plt.title("Pure Im Method")
plt.legend()
plt.grid(True)
plt.show()
